In [1]:
import torch
import torchaudio
from torchaudio.pipelines import MMS_FA as bundle
from typing import List
import IPython
import matplotlib.pyplot as plt
from pypinyin import pinyin, lazy_pinyin, Style

print(torch.__version__)
print(torchaudio.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import whisper
from pydub import AudioSegment
import string

2.1.2+cu118
2.1.2+cu118
cuda


In [2]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

raw_data = "raw_audio\\papaya.WAV"

model = whisper.load_model("large-v2")
# 使用Whisper進行語音識別
result = model.transcribe(raw_data)
result['segments']

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 1.6400000000000001,
  'text': 'Hi 大家好',
  'tokens': [50364, 17155, 220, 15248, 50446],
  'temperature': 0.0,
  'avg_logprob': -0.14041347129672183,
  'compression_ratio': 1.2415902140672783,
  'no_speech_prob': 0.0011157594854012132},
 {'id': 1,
  'seek': 0,
  'start': 1.6400000000000001,
  'end': 5.84,
  'text': '今天我們來聊聊一般人在初學 Excel 時常犯的幾個錯誤',
  'tokens': [50446,
   12074,
   5884,
   3763,
   40096,
   40096,
   2257,
   49640,
   4035,
   3581,
   28727,
   21372,
   19060,
   220,
   6611,
   11279,
   35276,
   107,
   1546,
   23575,
   3338,
   13133,
   3549,
   97,
   50656],
  'temperature': 0.0,
  'avg_logprob': -0.14041347129672183,
  'compression_ratio': 1.2415902140672783,
  'no_speech_prob': 0.0011157594854012132},
 {'id': 2,
  'seek': 0,
  'start': 5.84,
  'end': 9.200000000000001,
  'text': '以及這些問題有哪些理想的解決方案',
  'tokens': [50656,
   40282,
   29869,
   17197,
   2412,
   17028,
   13824,
   13876,
   7093,
   1546,
   17

In [12]:
sentence =[]
for i in range(len(result['segments'])):
    sentence.append([result['segments'][i]['text'].lower().replace('.','').replace(',', '').replace('6','六').replace('4','四').replace('2','二').replace('9','九').replace('8','八').replace('5','五').replace('3','三').replace('0','零').replace('1','一').replace('7','七').replace(' ',''), result['segments'][i]['start'], result['segments'][i]['end']])
    

sentence


[['hi大家好', 0.0, 1.6400000000000001],
 ['今天我們來聊聊一般人在初學excel時常犯的幾個錯誤', 1.6400000000000001, 5.84],
 ['以及這些問題有哪些理想的解決方案', 5.84, 9.200000000000001],
 ['那首先第一個錯誤是跨欄功能的過度使用', 9.200000000000001, 13.24],
 ['跨欄功能雖然可以讓表格產生更好的視覺效果', 13.24, 17.04],
 ['但是卻不利於後續的資料處理與分析', 17.04, 20.52],
 ['舉例來說', 20.52, 21.72],
 ['假設今天我想要用sum函數把這些金額組個加總', 21.72, 25.88],
 ['但你會發現在匡選金額的過程中', 25.88, 28.48],
 ['範圍會受到跨欄儲存格的干擾', 28.48, 31.36],
 ['那這個問題有兩個常見的解決方式', 31.36, 34.480000000000004],
 ['第一你可以在匡選時按住ctrl鍵不放', 34.480000000000004, 37.64],
 ['建立不相連的選取範圍來避開中間的標題', 37.64, 41.88],
 ['其次你也可以將這三個標題先解除他們的跨欄功能', 41.88, 46.36],
 ['使他們回復成一般的儲存格', 46.36, 49.08],
 ['然後再打開對齊方式的對話框', 49.08, 51.6],
 ['來從下拉選單中選取跨欄置中', 51.6, 54.84],
 ['雖然這兩個指令名稱乍看之下是相同的', 54.84, 58.24],
 ['但後者在置中文字標題時', 58.24, 60.52],
 ['其實仍保持儲存格為分離的狀態', 60.52, 63.64],
 ['因此當我們在用sum公式的時候', 63.64, 66.2],
 ['我們就可以一口氣從上方拉到表格底部', 66.2, 69.12],
 ['計算出所有產品的金額加總', 69.12, 72.0],
 ['好儘管這兩個方式都可以解決跨欄的問題', 72.0, 75.64],
 ['但如果你有打算對表格內的資料做進一步的分析', 75.64, 79.32000000000001],
 ['一般仍不

In [13]:
audio = AudioSegment.from_file("raw_audio\\papaya.wav")
for i in range(len(sentence)):
    audio_clip = audio[sentence[i][1] *1000: sentence[i][2]*1000]
    audio_clip.export(f"sentences\\{(sentence[i][0].lower())}.wav", format="wav")

In [14]:
model = bundle.get_model()
model.to(device)

tokenizer = bundle.get_tokenizer()
aligner = bundle.get_aligner()

def compute_alignments(waveform: torch.Tensor, transcript: List[str]):
    with torch.inference_mode():
        emission, _ = model(waveform.to(device))
        token_spans = aligner(emission[0], tokenizer(transcript))
    return emission, token_spans

def _score(spans):
    return sum(s.score * len(s) for s in spans) / sum(len(s) for s in spans)


def plot_alignments(waveform, token_spans, emission, transcript, sample_rate=bundle.sample_rate):
    ratio = waveform.size(1) / emission.size(1) / sample_rate

    fig, axes = plt.subplots(2, 1)
    axes[0].imshow(emission[0].detach().cpu().T, aspect="auto")
    axes[0].set_title("Emission")
    axes[0].set_xticks([])

    axes[1].specgram(waveform[0], Fs=sample_rate)
    for t_spans, chars in zip(token_spans, transcript):
        t0, t1 = t_spans[0].start, t_spans[-1].end
        axes[0].axvspan(t0 - 0.5, t1 - 0.5, facecolor="None", hatch="/", edgecolor="white")
        axes[1].axvspan(ratio * t0, ratio * t1, facecolor="None", hatch="/", edgecolor="white")
        axes[1].annotate(f"{_score(t_spans):.2f}", (ratio * t0, sample_rate * 0.51), annotation_clip=False)

        for span, char in zip(t_spans, chars):
            t0 = span.start * ratio
            axes[1].annotate(char, (t0, sample_rate * 0.55), annotation_clip=False)

    axes[1].set_xlabel("time [second]")
    fig.tight_layout()

def preview_word(waveform, spans, num_frames, transcript, sample_rate):
    ratio = waveform.size(1) / num_frames
    x0 = int(ratio * spans[0].start)
    x1 = int(ratio * spans[-1].end)
    #print(f"{transcript} ({_score(spans):.2f}): {x0 / sample_rate:.3f} - {x1 / sample_rate:.3f} sec")
    time_StarAndEnd = [ x0 / sample_rate, x1/sample_rate] # 回傳單個字的起始時間與結束時間
    segment = waveform[:, x0:x1]
    #return IPython.display.Audio(segment.numpy(), rate=sample_rate)
    return time_StarAndEnd
    

In [16]:
for i in range(1,len(sentence)):#len(sentence)-1
    text_normalized = ' '.join(lazy_pinyin(sentence[i][0]))#將文字轉為沒有音調的拼音，lazy_pinyin是陣列所以要再join成字串

    waveform, sample_rate = torchaudio.load(f"sentences\\{sentence[i][0]}.wav")
    waveform = waveform[0:1]

    transcript = text_normalized.split()
    emission, token_spans = compute_alignments(waveform, transcript)
    num_frames = emission.size(1)


    #plot_alignments(waveform, token_spans, emission, transcript)

    print("Raw Transcript: ", sentence[i][0])
    print("Normalized Transcript: ", text_normalized)
    IPython.display.Audio(waveform, rate=sample_rate)

    text_raw = sentence[i][0]
    word_start_end = []
    pinyin_tone = pinyin(text_raw, style=Style.TONE3, heteronym=False)
    for j in range(len(transcript)):#len(transcript)
        timeStartEnd = preview_word(waveform, token_spans[j], num_frames, transcript[j], sample_rate)
        word_start_end.append([pinyin_tone[j][0], timeStartEnd[0], timeStartEnd[1]])
    print(word_start_end)

    audio = AudioSegment.from_file(f"sentences\\{sentence[i][0]}.wav")
    file_name = sentence[i][0]
    for k in range(len(word_start_end)):
        segment_audio = audio[word_start_end[k][1] *1000: word_start_end[k][2]*1000]
        segment_audio.export(f"data\\{file_name}-{k}_{word_start_end[k][0]}.wav", format="wav")
    print('------------------------------------------')

Raw Transcript:  今天我們來聊聊一般人在初學excel時常犯的幾個錯誤
Normalized Transcript:  jin tian wo men lai liao liao yi ban ren zai chu xue excel shi chang fan de ji ge cuo wu
[['jin1', 0.0, 0.11625850340136054], ['tian1', 0.15258503401360543, 0.3560544217687075], ['wo3', 0.4214512471655329, 0.45049886621315194], ['men', 0.45049886621315194, 0.4941043083900227], ['lai2', 0.5304308390022676, 0.6103628117913832], ['liao2', 0.6248979591836735, 0.799297052154195], ['liao2', 0.8646938775510205, 0.9155555555555556], ['yi4', 0.9155555555555556, 0.9664172335600907], ['ban1', 1.126281179138322, 1.2207482993197278], ['ren2', 1.2280272108843537, 1.4314739229024944], ['zai4', 1.4823356009070294, 1.6131292517006803], ['chu1', 1.693061224489796, 1.743922902494331], ['xue2', 1.8020634920634921, 1.9619274376417233], ['excel', 2.129047619047619, 2.5359637188208617], ['shi2', 2.572312925170068, 2.8193650793650793], ['chang2', 2.84843537414966, 2.993764172335601], ['fan4', 3.0736961451247167, 3.2044897959183674], ['de', 3.